# Тестирование сервиса в ручную

In [1]:
from pathlib import Path

DATA_PATH = Path('../../data/')
DATA_PATH.mkdir(parents=True, exist_ok=True)

DATA_CACHE = DATA_PATH / Path('cache_dir/')
DATA_CACHE.mkdir(parents=True, exist_ok=True)

LOG_DATA = Path('../../logs/')
LOG_DATA.mkdir(parents=True, exist_ok=True)

import pandas as pd

pd.set_option('display.max_colwidth', 500) 

In [2]:
import sys
import os

project_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(project_path)

In [3]:
from app.settings import settings
from app.models.openai import OpenAIChatModel, OpenAIImageModel, OpenAIAudioModel

# Проверим API ключ
print(f"OpenAI API Key загружен: {'да' if settings.OPENAI_API_KEY else 'нет'}")


OpenAI API Key загружен: да


# Чат

In [4]:
llm = OpenAIChatModel(api_key=settings.OPENAI_API_KEY, model_name="gpt-4.1-nano")


In [5]:
messages = [{"role": "user", "content": "Привет, как тебя зовут?"}]
response = llm.chat(messages=messages)

print("Ответ модели:")
print(response)


2025-08-15 18:39:50 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Ответ модели:
Привет! Я — искусственный интеллект, и у меня нет имени, но ты можешь обращаться ко мне как к помощнику или выбрать любое имя. Как я могу помочь тебе сегодня?


In [6]:
messages = [{"role": "user", "content": "Напиши короткий рассказ про кота"}]

print("Потоковый ответ:")
for chunk in llm.chat_stream(messages=messages):
    print(chunk, end="", flush=True)


Потоковый ответ:
2025-08-15 18:39:51 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Жил-был в небольшом уютном доме серый кот по имени Мурчик. Он был очень любопытным и любил исследовать каждый уголок своего жилья. Одним солнечным утром Мурчик обнаружил в саду красивую пёструю бабочку. Она порхала из цветка в цветок, заманивая кота за собой. Мурчик тихо подкрался и чуть не схватил её, но бабочка вдруг взмыла в воздух и улетела. Тогда кот понял, что иногда самое приятное — просто наблюдать за красивым и свободным миром вокруг. И с тех пор он стал чаще гулять в саду, наслаждаясь каждым моментом.

In [9]:
from app.services.token_manager import TokenManager

tm = TokenManager()
text = "Привет, мир!"
print(f"Текст: {text}")
print(f"Токенов: {tm.count_tokens(text, model_name='gpt-4.1-nano')}")


Текст: Привет, мир!
Токенов: 5


# Аудео

In [10]:
import io

audio_model = OpenAIAudioModel(
    api_key=settings.OPENAI_API_KEY,
    model_name="gpt-4o-mini-transcribe"
)

audio_path = Path("test.wav")
with open(audio_path, "rb") as f:
    audio_stream = io.BytesIO(f.read())

In [11]:
result_text = audio_model.transcribe(audio_stream, filename=audio_path.name)
print("Результат транскрипции:")
print(result_text)


2025-08-15 18:40:20 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/audio/transcriptions "HTTP/1.1 200 OK"
Результат транскрипции:
Шла Саша по шоссе и сосала сушку


# Изображения

In [4]:
import base64
from pathlib import Path
from app.models.openai import OpenAIImageModel
from app.settings import settings

image_model = OpenAIImageModel(
    api_key=settings.OPENAI_API_KEY,
    model_name="gpt-image-1"
)

prompt = "Футуристический город на закате"
result_images = image_model.generate(
    prompt=prompt,
    n=1,
    size="1024x1024",
    quality="low"
)

2025-08-15 18:43:04 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/images/generations "HTTP/1.1 403 Forbidden"


PermissionDeniedError: Error code: 403 - {'error': {'message': 'Your organization must be verified to use the model `gpt-image-1`. Please go to: https://platform.openai.com/settings/organization/general and click on Verify Organization. If you just verified, it can take up to 15 minutes for access to propagate.', 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [ ]:

output_dir = Path("generated_images")
output_dir.mkdir(exist_ok=True)

for idx, img_b64 in enumerate(result_images, start=1):
    img_bytes = base64.b64decode(img_b64)
    img_path = output_dir / f"image_{idx}.png"
    with open(img_path, "wb") as f:
        f.write(img_bytes)
    print(f"✅ Сохранено: {img_path}")
